In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_calibration_experiments

fname = 'results/calibration.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    print('could not read', fname)
    df_total = parse_calibration_experiments()
    pd.to_pickle(df_total, fname)
    print('saved as', fname)

In [ ]:
df_total.tail()